In [ ]:
import matplotlib.pyplot as plt
from env import DATABASE_URI
from sqlalchemy import create_engine, select, and_
from sqlalchemy.orm import Session, scoped_session, sessionmaker
import models as md
from matplotlib import colormaps
import numpy as np
engine = create_engine(DATABASE_URI)
import pandas as pd

In [ ]:
# List runs
with Session(engine) as session:
    run_query = select(md.Run).where(md.Run.mode == "REAL")
    print(session.execute(run_query.order_by(md.Run.id)).scalars().all())

In [ ]:
# Plot over cooldown


def query_bb(run_id:int, path_id:int, cut_in=0) -> tuple[list, list]:
    with Session(engine) as session:
        bb_data_query = select(md.BbResistancePathData).where(and_(md.BbResistancePathData.run_id == run_id, md.BbResistancePathData.path_id == path_id))
        data = session.execute(bb_data_query).scalars().all()

        t0 = data[cut_in].timestamp
        elapsed_time = lambda t: (t - t0).total_seconds() / (60*60)

        times = [elapsed_time(row.timestamp) for row in data]
        temps = [row.ohms for row in data]
        return times[cut_in:], temps[cut_in:]

def query_sensor(run_id: int, sensor: str, cut_in=0) -> tuple[list, list]:
    with Session(engine) as session:
        query = select(md.Data).where(and_(md.Data.run_id == run_id, md.Data.sensor == sensor))
        data = session.execute(query.order_by(md.Data.id)).scalars().all()
        if not data:
            print("no data found!")
            return
        t0 = data[cut_in].timestamp
        elapsed_time = lambda t: (t - t0).total_seconds() / (60*60) 

        times = [elapsed_time(row.timestamp) for row in data]
        temps = [row.celcius for row in data]

        print(f"last reading was at {data[-1].timestamp}")
        return times[cut_in:], temps[cut_in:]


run_id = 92

color = 'red'
sens_num = 4
for i in range(1,5):
    sens_num = i 
    fig, ax_ohms = plt.subplots(figsize=(10,10))

    ax_ohms.set_xlabel('time (hours)')
    ax_ohms.set_ylabel('Resistance (Ohms)', color=color)
    ax_ohms.tick_params(axis='y', labelcolor=color)
    ax_ohms.scatter(*query_bb(run_id, sens_num), color=color, s=4, label=str(sens_num))
    ax_ohms.set_yticks(range(0,1100,100))
    ax_ohms.legend(title="Resistance Path", shadow=True, fontsize=14, title_fontsize=16)
    ax_ohms.grid()

    ax_T = ax_ohms.twinx()  # instantiate a second Axes that shares the same x-axis

    ax_T.set_ylabel('Temperature (Celcius)', color="blue")
    ax_T.tick_params(axis='y', labelcolor="blue")
    ax_T.plot(*query_sensor(run_id, f"L{sens_num}"), color="blue", linestyle="solid", label=f"L{sens_num}")
    ax_T.plot(*query_sensor(run_id, f"E{sens_num}"), color="mediumblue", linestyle="solid", label=f"E{sens_num}")
    ax_T.set_ylim(-35, 20)  # Set the y-axis range for temperature

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    ax_T.legend(title="Temperature", loc='center right', shadow=True, fontsize=14, title_fontsize=16)

    plt.title(f"Overview of Temperature and BB Health: Position {sens_num}")


In [ ]:
## Bringing in the CO2 information

# Year/Month/Day Hour:Minute:Second.Millisecond
df_co2 = pd.read_csv("./CO2 temperature.csv", usecols=range(3), parse_dates=['Time'])
df_co2.ffill(inplace=True)
df_co2['Time'] = pd.to_datetime(df_co2['Time'])  # Ensure 'Time' is in datetime format
df_co2['Time'] = df_co2['Time'].dt.tz_localize('Europe/Zurich')  # Localize to Geneva timezone
df_co2.sort_values('Time', inplace=True)  # Ensure df_co2 is sorted

run_id = 85
sensors = ["L1", "L2", "L3", "L4", "E1", "E2", "E3", "E4", "P1", "P2", "P3"]
with Session(engine) as session:
    tm_data = {}
    for sensor in sensors:
        query = select(md.Data).where(and_(md.Data.run_id == run_id, md.Data.sensor == sensor))
        data = session.execute(query.order_by(md.Data.id)).scalars().all()

        sensor_data = {}
        sensor_data[f"{sensor}_Temperature"] = [row.celcius for row in data]
        sensor_data[f"{sensor}_Time"]        = [row.timestamp for row in data]

        tm_data[sensor] = pd.DataFrame(sensor_data)

df = df_co2.copy()
for sensor, sensor_df in tm_data.items():
    sensor_df.sort_values(f"{sensor}_Time", inplace=True)
    sensor_df[f"{sensor}_Time"] = sensor_df[f"{sensor}_Time"].dt.tz_convert('Europe/Zurich')

    df = pd.merge_asof(
        df,
        sensor_df,
        left_on='Time',
        right_on=f"{sensor}_Time",
        direction='nearest',
        tolerance = pd.Timedelta('30s')
    )
    del df[f"{sensor}_Time"]


In [ ]:
df['Elapsed'] = (df['Time'] - df["Time"].iloc[0]).dt.total_seconds() / (60*60)
# plt.plot(df['Elapsed'], df['CO2_Temperature_Outlet'], label='Outlet Temperature')
# plt.plot(df['Elapsed'], df['CO2_Temperature_Inlet'] , label='Inlet Temperature')
# plt.plot(df['Elapsed'], df['L1_Temperature'] , label='Inlet Temperature')

# plt.plot(df["Elapsed"], df["L1_Temperature"]-df["CO2_Temperature_Outlet"])

df2 = df[(df["Elapsed"] > 5.60) & (df["Elapsed"] < 5.8)]
# df2 = df
plt.figure(figsize=(10,10))
plt.title("No Power to 6W")
plt.plot(df2["Elapsed"], df2["L4_Temperature"], label="LGAD 4")
plt.plot(df2["Elapsed"], df2["L3_Temperature"], label="LGAD 3")
plt.plot(df2["Elapsed"], df2["L2_Temperature"], label="LGAD 2")
plt.plot(df2["Elapsed"], df2["L1_Temperature"], label="LGAD 1")

plt.plot(df2["Elapsed"], df2["E4_Temperature"], label="ETROC 4")
plt.plot(df2["Elapsed"], df2["E3_Temperature"], label="ETROC 3")
plt.plot(df2["Elapsed"], df2["E2_Temperature"], label="ETROC 2")
plt.plot(df2["Elapsed"], df2["E1_Temperature"], label="ETROC 1")

plt.plot(df2["Elapsed"], df2["P3_Temperature"], label="Probe 3")
plt.plot(df2["Elapsed"], df2["P2_Temperature"], label="Probe 2")
plt.plot(df2["Elapsed"], df2["P1_Temperature"], label="Probe 1")
plt.plot(df2["Elapsed"], df2["CO2_Temperature_Outlet"], label="CO2 Outlet", linestyle="--")
plt.plot(df2["Elapsed"], df2["CO2_Temperature_Inlet"], label="CO2 Inlet", linestyle="--")
plt.xlabel("Time (hours)")
plt.ylabel("Temperature (Celcius)")


# plt.axvline(4, color="slategrey", label="0W", linestyle="-.")
# plt.axvline(4.2, label="1W")
# plt.axvline(4.35, label="2W")
# plt.axvline(4.6, label="3W")
# plt.axvline(4.75, label="4W")
# plt.axvline(4.9, color="darkseagreen", label="5W", linestyle="-.")
# plt.axvline(5.1, label="6W")
# plt.axvline(5.25, label="7W")
# plt.axvline(5.4, color="salmon", label="8W", linestyle="-.")


plt.legend()
plt.grid()


# near_4_9 = df2[(df2['Elapsed'] >= 4.87) & (df2['Elapsed'] <= 4.93)]
# print(np.mean(near_4_9["P2_Temperature"]-near_4_9["L4_Temperature"]))

In [ ]:
run_id = 79


times, temps = query_sensor(run_id, "L1")
times = np.array(times)
temps = np.array(temps)

mask = (times <= 310) & (times >= 135)
plt.plot(times[mask] - times[mask][0], temps[mask])

print((times[mask][-1] - times[mask][0])/60)
print(temps[mask][-1] - temps[mask][0])

plt.grid()
plt.xlabel("Time (minutes)")
plt.ylabel("Temperature (Celcius)")
plt.title("Cooldown of TM2")